# Chapter 3. 딕셔너리와 집합
- `dict`형은 어플리케이션에서 널리 사용되는 파이썬 구현의 핵심부
- 파이썬 `dict` 클래스는 **해시 테이블**을 통해 최적화 되어 있음
- 집합도 **해시 테이블** 이용해 구현하므로, 둘을 함께 다루게 됨


## 3.1 일반적인 매핑형
- `collections.abc` 모듈은 `dict` 및 이와 유사한 자료형의 인터페이스를 정의하기 위해 `Mapping` 및 `MutableMappting` 추상 베이스 클래스를 제공
- 특화된 매핑은 추상 베이스 클래스 대신 `dict`나 `collections.UserDict` 클래스를 상속하기도 함

In [1]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

- 표준 라이브러리에서 제공하는 매핑형은 모두 `dict`를 이용해서 구현하므로, 키가 **해시 가능**해야 한다는 제한이 존재
- `str`, `byte`, 수치형 등의 원자 자료형은 모두 해시 가능
- 튜플은 들어 있는 항목들이 모두 해시 가능해야 해시 가능

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [3]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [4]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

985328935373711578

- 사용자 정의 자료형은 기본적으로 해시 가능
- 객체의 해시값은 `id()`를 이용해 구하는데 이는 모든 객체의 id가 서로 다르기 때문
- 객체가 `__eq__()` 메서드를 직접 구현하는 경우, 해시값 계산에 사용되는 속성이 모두 불변형일 때만 해시 가능

In [7]:
# dict를 구현하는 다양한 방법
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

## 3.2 지능형 딕셔너리
**dictcom**는 모든 반복형 객체에서 키-값 쌍을 생성함으로써 딕셔너리 객체 생성 

In [9]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia')
]

country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62}

In [11]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 62: 'INDONESIA'}

## 3.3. 공통적인 매핑 메서드
`dict`의 변형 중 가장 널리 사용되는 클래스는 `defaultdict`와 `OrderedDict`

- `update()` 메서드가 첫 번째 인수 `m`을 다루는 방식은 **덕 타이핑**의 대표적 사례
- `m`이 `keys()` 메서드를 갖고 있다면 매핑이라고 간주, 메서드를 가지고 있지 않다면 `update()` 메서드는 `m`의 항목들이 (키, 값) 쌍으로 되어 있다 간주하고 `m`을 반복
- 대부분의 파이썬 매핑은 `update()` 메서드와 같은 논리를 내부적으로 구현

### 3.3.1 존재하지 않는 키를 setdefault( )로 처리하기
- **fail-fast** 철학에 따라, 존재하지 않는 키 `k`로 `d[k]`를 접근하면 오류 발생
- **KeyError**를 처리하는 것보다 기본 값을 사용하는 것이 편리한 경우 `d[k]` 대신 `d.get(k, default)`를 사용하는 것이 좋으나, 아래 예제는 `dict.get()`의 나쁜 사례를 보여줌

In [12]:
"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""

import sys
import re

WORD_RE = re.compile(r'\w+')
index = {}

with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            
            occurrences = index.get(word, []) # 리스트가 없으면 빈 배열 설정
            occurrences.append(location)  # 배열에 location 튜플 추가
            index[word] = occurrences  # 추가된 리스트를 다시 할당

for word in sorted(index, key=str.upper):
    print(word, index[word])

FileNotFoundError: [Errno 2] No such file or directory: '-f'

- 위 예제는 `index`를 두 번이나 검색해야 하는 비효율이 있음
- `occurrences`를 처리하는 코드 세 줄을 `dict.setdefault()`를 사용하면 한 줄로 작성 가능하며, `index`를 한 번만 검색할 수 있게 됨

In [13]:
import sys
import re

WORD_RE = re.compile(r'\w+')
index = {}

with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            
            index.setdeafult(word, []).append(location)  # 리스트 없으면, 빈 배열

for word in sorted(index, key=str.upper):
    print(word, index[word])

FileNotFoundError: [Errno 2] No such file or directory: '-f'

## 3.4 융통성 있게 키를 조회하는 매핑
- 검색할 때 키가 존재하지 않으면 특별한 값을 반환하는 매핑이 있으면 편리
- 이를 구현하는 방법은 두 가지가 있음
    1. `defaultdict`를 사용
    2. `dict` 등의 매핑형을 상속해 `__missing__()` 메서드를 추가
    
### 3.4.1 defaultdict: 존재하지 않는 키에 대한 또 다른 처리
- `defaultdict`는 존재하지 않는 키로 검색할 때 요청에 따라 항목을 생성
- 작동 방식은 `defaultdict` 객체를 생성할 때, 존재하지 않는 키 인수로 `__getitem__` 메서드를 호출할 때마다 기본값을 생성하기 위해 사용되는 콜러블을 제공하는 것
- e.g. `dd = defaultdict(list)` 의 선언 후, `dd['new-key']`로 접근을 하면 다음과 같이 처리
    1. 리스트를 새로 생성하기 위해 `list()`를 호출: 콜러블
    2. `'new-key'`를 키로 사용해 새로운 리스트를 `dd`에 삽입
    3. 리스트에 대한 참조를 반환
- 기본값을 생성하는 콜럽르은 `default_factory` 객체 속성에 저장됨

In [ ]:
import sys
import re
import collections

WORD_RE = re.compile(r'\w+')
index = collections.defaultdict(list)

with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index[word].append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

### 3.4.2 __missing__( ) 메서드
- 매핑형은 `__missing__()` 메서드를 이용해 존재하지 않는 키를 처리
- 이는 `dict`에 정의되어 있지만 `dict`는 해당 메서드를 알고 있음
- 따라서 `dict` 클래스를 상속하고 `__missing__()` 메서드를 정의하면, `dict.__getitem__()` 표준 메서드가 키를 발견할 수 없을 때 `__missing__()` 메서드를 호출
- 비문자열 키를 검색할 때 키를 발견하지 못하면 **키를 문자열로 변환**하는 객체를 구현해볼 것

In [23]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        # 일종의 베이스 컨디션
        if isinstance(key, str):  # 키가 문자열임에도 존재하지 않으면 에러
            raise KeyError(key)
        return self[str(key)]     # 아니라면, 문자열로 변환해 원소 반환
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

In [24]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [25]:
d[4]

'four'

In [26]:
d[1]

KeyError: '1'

In [27]:
d.get('2')

'two'

In [28]:
d.get(4)

'four'

In [29]:
d.get(1, 'N/A')

'N/A'

In [30]:
2 in d

True

In [31]:
1 in d

False

- 모든 연산에 `self[key]` 부분을 활용해 `__missing__()` 메서드가 호출되도록 함
- 위 코드에서는 일관성 있는 동작을 구성하기 위해 `__contains__()` 메서드도 재구현하는데, 이를 구현하지 않으면 원래의 `__contains__()` 메서드가 `__missing__()`을 호출하지 않기 때문

```
# Bad
def __contains__(self, key):
    return key in self or str(key) in self

# Good
def __contains__(self, key):
    return key in self.keys() or str(key) in self.keys()
```

- 위 예제처럼 `key in self` 와 같이 조회를 하게 되면, 재귀적으로 `__contains__()` 메서드를 호출하기 때문에 이러한 문제를 피하기 위해 `key in self.keys()`와 같이 구현

## 3.5 그 외 매핑형

#### collections.OrderedDict
- 키를 삽입한 순서대로 유지함으로써 항목을 반복하는 순서를 예측 가능
- `popitem()` 메서드는 최근에 삽입한 항목을 꺼내지만, `popitem(last=True)` 형태로 호출하면 처음 삽입한 항목을 꺼냄

#### collections.ChainMap
- 매핑들의 목록을 담고 있으며 한꺼번에 모두 검색 가능
- 각 매핑을 차례대로 검색하고, 그중 하나에서라도 키가 검색되면 성공
- 아래 코드는 파이썬에서 변수를 조회하는 기본 규칙

```
import builtins
pylookup = ChainMap(locals(), globals(), vars(builtins))
```

#### collections.Counter
- 모든 키에 정수형 카운터를 지니는 매핑으로, 기존 키를 갱신하면 카운터가 증가
- 합게를 구하기 위한 `+` 와 `-` 연산자를 구현하며, _n_ 개의 가장 널리 사용된 항목과 그들의 카운터로 구성된 튜플 리스트를 반환하는 `most_common([n])` 메서드를 제공

```
ct = collections.Counter('abracadabra')
ct.update('aaaaazzz')
ct.most_common(2)
```

#### collections.UserDict
- 표준 `dict` 철머 작동하는 매핑을 순수 파이썬으로 구현한 클래스

## 3.6 UserDict 상속하기
- `dict` 보다는 `UserDict`를 상속해 매핑형을 만드는 것이 쉬움
- `UserDict`는 `dict`를 상속하지 않고 내부에 실제 항목을 담고 있는 `data`라고 하는 `dict` 객체를 지님
- 이렇게 구현함으로써 `__setitem__()` 등의 특수 메서들르 구현할 때 발생하는 원치 않는 재귀 호출을 피할 수 있으며, `__contains__()` 메서드도 간단히 구현 가능
- 아래 구현할 `StrKeyDict`는 몯느 키를 `str` 형으로 저장함으로서 비문자열 키로 객체를 생성하거나 갱신할 때 발생할 수 있는 예기치 못한 문제를 피하게 해줌

In [32]:
import collections

class StrKeyDict(collections.UserDict):

    def __missing__(self, key):
        if isinstance(self, key):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data  # 저장된 키가 모두 str 이므로 바로 조회 가능
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item  # 모든 키를 str 형으로 변환

`UserDict` 클래스가 `MutableMapping`을 상속하므로 `StrKeyDict`는 결국 `UserDict`, `MutableMapping` 또는 `Mapping`을 상속하게 되어 매핑의 모든 기능을 지니게 됨

## 3.7 불변 매핑
- 표준 라이브러리에서 제공하는 매핑형은 모두 가변형이지만, 불변의 설정도 가능
- `types` 모듈의 `MappingProxyType` 이라는 래퍼 클래스는 원래 매핑의 동적 뷰를 제공하지만 **read-only**의 `mappingproxy` 객체를 반환
- 따라서 원래 매핑을 변경하면 `mappingproxy`에 반영되지만, `mappingproxy`를 직접 변경할 수는 없음

In [33]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [34]:
d_proxy[1]

'A'

In [35]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [37]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [38]:
d_proxy[2]

'B'

## 3.8 집합 이론